# Machine Learning 2018.3
> - **Lista 5** - *2ª entrega (no dia da 2ª prova)*
- **Lucas Lopes Felipe** - *PPGI*
- **Turma**: *CPS-863 / COS-623 / MAB-608*
- **Professores**: Edmundo, Daniel S. Menascé e Rosa Leão

## Questão 1

> Hidden Markov models have been widely used in the area of speech recognition. Your job is to search the literature for a _simple_ application example (except those that were shown in class) in this area and to describe _concisely_ such example, indicating the basic ideas used, presenting the model and what is done to obtain the solution. For this assignment, it is your job to summarize and write **no more than a page** to clearly demonstrate that you understand the example. There is plenty of material about this topic in the literature including the main references used in this class. You should use those and/or look for additional references and indicate the source of your material. Your draft should include:

- a sample (HMM) model used in the application.
- how the model is useful to accomplish the task you describe.
- from the problems given in class (_problem 1, 2_, etc) what are those used to solve the problem.

## Questão 2

> In class we described the coin tossing toy example summarized in what follows. Suppose that a person is tossing a coin using $2$ biased coins. Assume you know that the person changes from one coin to another with probability $0.2$ after each coin flipping. Unfortunately, you do not know the probabilities that each coin flipping results in a head ($H$) or a tail ($T$).

> Your job in this question is to calculate the probability of observing a given sequence of heads and tails. Since you know nothing about the coins, you build a simple Markov model that should, you hope, explain the sequence of observations. The model is shown below:

<img src="./imgs/markov-coin.png" width="400"/>

In [1]:
import numpy as np

def matriz_transicao(p_H):
    # prob_t to [Head and Tail]
    coin = [p_H, 1 - p_H]
    return np.array([coin, coin])

def matriz_pi(num_estados):
    prob_identica = 1 / num_estados
    return np.repeat(prob_identica, num_estados)

def markov_prob(trans, pi, amostra):
    inicio = (amostra[0] == 'T') * 1
    prob   = pi[inicio]
    
    for i in range(1, len(amostra)):
        row   = (amostra[i - 1] == 'T') * 1
        col   = (amostra[i]     == 'T') * 1
        prob *= trans[row, col]
        
    return prob

> Suppose that you observe the sequence $HHHHHTTTTTT$ and you decide to assign $P[H] = 0.5$ to your model.

In [2]:
m_transicao    = matriz_transicao(0.5)
m_estacionaria = matriz_pi(2)

1. **From the model, calculate the probability that the sequence $HHHHHTTTTTT$ is observed.**

In [3]:
seq_1  = 'HHHHHTTTTTT'
prob_1 = markov_prob(m_transicao, m_estacionaria, seq_1)

print('\nA probabilidade da sequência HHHHHTTTTTT',
      'ter sido gerado tal que P[H] = 0.5 é de:', prob_1)


A probabilidade da sequência HHHHHTTTTTT ter sido gerado tal que P[H] = 0.5 é de: 0.00048828125


2. **Repeat your calculations if, instead, you observe the sequence $HTHTHTHTHTH$.**

In [4]:
seq_2  = 'HTHTHTHTHTH'
prob_2 = markov_prob(m_transicao, m_estacionaria, seq_2)

print('\nA probabilidade da sequência HTHTHTHTHTH',
      'ter sido gerado tal que P[H] = 0.5 é de:',prob_2)


A probabilidade da sequência HTHTHTHTHTH ter sido gerado tal que P[H] = 0.5 é de: 0.00048828125


3. **Since you do not know if $P[H] = 0.5$ is a reasonable value to use, your job is to obtain $P[H]$, which maximizes the likelihood $(\; P(\mathcal{D}\mid \mathcal{M})\;)$, for each of the sequences above.**

A probabilidade de ir de um estado $i$ para outro $j$ é obtido através do número de vezes em que houve transição de $i$ para $j$ dividido pelo total de vezes que saiu-se do estado $i$ para qualquer outro estado. Que pode ser representando pela seguinte [fórmula](https://www.stat.cmu.edu/~cshalizi/462/lectures/06/markov-mle.pdf):

$$p_{ij} = \frac{n_{ij}}{\sum_{j = 1}^{s} n_{i,j}}$$

Onde $n_{i,j}$ representa a quantidade de vezer que foi-se do estado $i$ para $j$. E $s$ é o número de estados existentes na cadeia.

Como neste caso, todas as probabilidades de transição são em função da probabilidade de $P[H]$, basta dividir o número de vezes que $H$ aparece (exceto se este começar a sequência, pois não será uma transição e sim probabilidade estacionária), pelo tamanho da total da sequência $-1$ (total de transações ocorridas).

Como desmontrado na seguinte função:

In [5]:
def mle(amostra):
    Hs = 0
    if amostra[0] == 'H': Hs -= 1
    for a in amostra:
        if a == 'H': Hs += 1
    return Hs / (len(amostra) - 1)

Agora vamos achar o MLE ($P[H]$ ideal) para cada sequência.

In [6]:
mle_1   = mle(seq_1)
trans_1 = matriz_transicao(mle_1)

print('\nSequência 1:'
      '\nP[H] =', mle_1,
      '\nLikelihood =', markov_prob(trans_1, m_estacionaria, seq_1))


Sequência 1:
P[H] = 0.4 
Likelihood = 0.0005971968


In [7]:
mle_2   = mle(seq_2)
trans_2 = matriz_transicao(mle_2)

print('\nSequência 2:'
      '\nP[H] =', mle_2,
      '\nLikelihood =', markov_prob(trans_2, m_estacionaria, seq_2))


Sequência 2:
P[H] = 0.5 
Likelihood = 0.00048828125


4. **Obtain the value for $P[H]$ that maximizes the likelihood for the sequence $HTHTHTTT$.**

In [8]:
seq_3   = 'HTHTHTTT'
mle_3   = mle(seq_3)
trans_3 = matriz_transicao(mle_3)

print('\nSequência 3:'
      '\nP[H] =', mle_3,
      '\nLikelihood =', markov_prob(trans_3, m_estacionaria, seq_3))


Sequência 3:
P[H] = 0.2857142857142857 
Likelihood = 0.007589160493137578


## Questão 3

> Consider the Markov chain given below, that models some system. Suppose you observe the following sequence of states: $1, 2, 1, 3$ at instants $t_1, t_2, t_3, t_4$.

<img src="./imgs/markov.png" width="400"/>

In [9]:
# Estados a cada momento t
t = [0, 1, 2, 1, 3]

#    [  s1,   s2,   s3,   s4]
s1 = [0.60, 0.08, 0.02, 0.30]
s2 = [0.40, 0.20, 0.30, 0.10]
s3 = [0.20, 0.30, 0.00, 0.50]
s4 = [1.00, 0.00, 0.00, 0.00]

m_transicao = np.array([s1, s2, s3, s4])

1. **What is the most probable state that you should observe at instant $t_5$? (In this item assume that there is not data observed after $t_4$.)**

Devido a _Propriedade de Markov_ em que o próximo estado depende apenas do atual, o estado mais provável de ser visitado no instante $t_5$ será o que possui maior probabilidade a partir do estado que está no instante anterior $(t_4)$, ou seja, o estado $3$.

O estado $3$ pode ir para tais estados com as seguintes probabilidades:

- $3 \rightarrow  1 = 0.02$
- $3 \rightarrow  2 = 0.3$
- $3 \rightarrow  4 = 0.5$

Portanto, o estado mais provável na sequência, partido do $3$, é o estado $4$.

In [10]:
presente = t[4]
# O estado que está no presente é o valor que está em t[4] (t[0] = 0)

pos = presente - 1
# Subtrai-se 1 pois o estado S está na linha S - 1 (index começa em 0)

estado_provavel = m_transicao[pos].argmax()
# A coluna que possui o maior valor na linha do estado presente será o mais provável

estado_provavel += 1
# Soma-se 1 pois o estado da coluna C é o estado C + 1 (index começa em 0)

print('O estado mais provável a partir do estado',
      t[4], 'é o estado:', estado_provavel)

O estado mais provável a partir do estado 3 é o estado: 4


2. **What is the most probable state that you should observe at instant $t_6$? (In this item assume that there is not data observed after $t_4$.)**

Novamente, se quisermos responder qual o estado mais provável no instante $t_6$ precisamos olhar para o mais provável a partir do estado anterior, que está em $t_5$.

Porém, há apenas dados observados até $t_4$, portanto, para inferir qual o mais provável em $t_6$ precisamos multiplicar:

- Probabilidade de ir do estado em $t_4$ (estado $3$) para $t_5$ (os possíveis estados saindo do 3); pela:
- Probabilidade de transição entre este possível estado em $t_5$ e seu respectivo mais próvavel estado de destido em $t_6$.

O par de estados em $t_4$ e $t_5$ que resultar no maior produtório $prob(t_4 \rightarrow t_5) * prob(t_5 \rightarrow t_6)$, será a sequência mais provável.

Os estados que podem sair do estado $3$ e suas respetivas probabilidades são:

- $prob(3 \rightarrow  1) = 0.02$
- $prob(3 \rightarrow  2) = 0.3$
- $prob(3 \rightarrow  4) = 0.5$

Agora vamos pegar o mais provável a partir de cada um destes, bem como sua probabilidade:

- $prob(1 \rightarrow  1) = 0.6$
- $prob(2 \rightarrow  2) = 0.4$
- $prob(4 \rightarrow  1) = 1.0$

Basta multiplicar e ver a combinação de maior probabilidade:

- $prob(3 \rightarrow  1 \rightarrow  1) = 0.02 * 0.6 = 0.012$
- $prob(3 \rightarrow  2 \rightarrow  2) = 0.30 * 0.4 = 0.12$
- $prob(3 \rightarrow  4 \rightarrow  1) = 0.50 * 1.0 = 0.5$

Logo, a sequência mais provável é:
$t_4 = 3 \rightarrow t_5 = 4 \rightarrow t_6 = 1$ com prabilidade igual a $0.5$

3. **Imagine that you keep observing the system but you miss 1 observation. That is, you observe: $1, 2, 1, 3, X, 2, 1$**

**a)** _Is the most probable state you guessed for instant $t_5$ (above) the answer for this question? (yes/no). Justify your answer._

Não.

Na questão anterior precisávamos considerar apenas os prováveis estados a partir de $t_4 = 3$, escolhendo aquele de maior probabilidade de transição que é $t_5 = 4$.

Agora, por sabermos que o estado posterior à $t_5$ é $t_6 = 2$, precisamos considerar a probabilidade não somente de $t_4 \rightarrow t_5$ mas também de $t_5 \rightarrow t_6$. Multiplicando as probabilidade de transição de ($t_4 = 3 \rightarrow t_5 = X) * (t_5 = X \rightarrow t_6 = 2$). Onde $X$ são todos os estados possíveis que saem do estado $3$ e possam ir em seguida para o estado $2$.

**b)** _Show how to obtain the most probable missing state observation._

O estado perdido $X$ precede do estado $3$ e antecede do $2$, portanto os possíveis estados estão na intercessão entre:

Estados possíveis a partir de $3$:

- $3 \rightarrow  1 = 0.02$
- $3 \rightarrow  2 = 0.3$
- $3 \rightarrow  4 = 0.5$

Estados possíveis com destino ao $2$:

- $1 \rightarrow  2 = 0.08$
- $2 \rightarrow  2 = 0.2$
- $3 \rightarrow  2 = 0.3$

Os estados que pertencem a estes 2 grupos descritos acima são:

- $3 \rightarrow  X \rightarrow  2$
- $3 \rightarrow  1 \rightarrow  2 = 0.02 * 0.08 = 0.0016$
- $3 \rightarrow  2 \rightarrow  2 = 0.30 * 0.20 = 0.06$

Logo, o mais provável é: $X = 2$ com probabilidade $0.06$

## Questão 4

> Refer to _Question 2_. A friend of yours said that she knows the bias of each coin. She said that, if one coin is tossed, the result is head with probability $0.9$ (refer to this coin as coin $A$). The other coin ($B$) has only a $0.1$ probability of resulting a head when tossed. In addition, you know that the person that tosses the coins changes from one coin to another with probability $0.7$, after showing you the result.

<img src="./imgs/hmm-4.png" width="400"/>

> Since you have just learned about HMMs, you build a HMM and use the learned material to calculate the probability that each of the above sequences (sequence $HHHHHTTTTTT$ and $HTHTHTHTHTH$) is observed. Since you do not know which coin is tossed first your initial guess is that each coin is equally probable of being used for the first toss. To answer the questions below you may either implement the appropriate recursions your learned in class or **use the MTK module of the Tangram-II tool** (the manual is available), or an appropriate software package for HMMs (R for instance).

In [11]:
sq_1 = 'HHHHHTTTTTT'
sq_2 = 'HTHTHTHTHTH'

#     [  A,   B] 
a_t = [0.3, 0.7]
b_t = [0.7, 0.3]

#     [  H,   T] 
a_e = [0.9, 0.1]
b_e = [0.1, 0.9]

m_transicao = np.array([a_t, b_t])
m_emissao   = np.array([a_e, b_e])
m_inicio    = matriz_pi(2)

1. **Describe your calculations and compare the results with that obtained from the Markovian model. In other words, you should indicate which of the 2 models (MC or HMM) is the best to explain each of the 2 sequences above.**

In [12]:
def forward(A, B, pi, amostra):
    T = len(amostra)
    N = A.shape[0]
    
    alpha = np.zeros((T, N))
    
    pos = (amostra[0] == 'T') * 1
    alpha[0] = pi * B[:, pos]
    
    for t in range(1, T):
        pos = (amostra[t] == 'T') * 1
        alpha[t] = alpha[t-1].dot(A) * B[:, pos]
        
    likelihood = alpha[-1].sum()
    
    return alpha, likelihood

In [13]:
forward(m_transicao, m_emissao, m_inicio, sq_1)[1]

5.92535142432e-05

In [14]:
forward(m_transicao, m_emissao, m_inicio, sq_2)[1]

0.005977272905936001

2. **Calculate the probability that coin $A$ is the last one being tossed.**

In [74]:
def prob_ultimo(alpha, coin):
    pos = (coin == 'B') * 1
    return alpha[-1,pos] / alpha[-1].sum()

In [76]:
alpha_1 = forward(m_transicao, m_emissao, m_inicio, sq_1)[0]
alpha_2 = forward(m_transicao, m_emissao, m_inicio, sq_2)[0]

print('Probabilidade da moeda A ser a última da sequência', sq_1,
     'é de:', prob_ultimo(alpha_1, 'A'))

print('Probabilidade da moeda A ser a última da sequência', sq_2,
     'é de:', prob_ultimo(alpha_2, 'A'))

Probabilidade da moeda A ser a última da sequência HHHHHTTTTTT é de: 0.16216421630895447
Probabilidade da moeda A ser a última da sequência HTHTHTHTHTH é de: 0.9503400731145438


In [78]:
def backward(A, B, pi, amostra):
    T = len(amostra)
    N = len(A[0])

    beta = np.zeros((N,T))
    
    beta[:,-1] = 1
    
    for t in reversed(range(T - 1)):

        pos = (amostra[t + 1] == 'T') * 1
        for n in range(N):
            beta[n,t] = np.sum(A[n,:] * beta[:,t + 1] * B[:,pos])
    
    return beta


back = backward(m_transicao, m_emissao, m_inicio, sq_1)

back[:,1].sum()

0.0007746319008

In [68]:
def gamma(A, B, pi, amostra):
    alpha = forward(A, B, pi, amostra)[0]
    beta  = backward(A, B, pi, amostra)
    
    likelihood = forward(A, B, pi, amostra)[1]

    return np.dot(alpha, beta) / likelihood

gamma(m_transicao, m_emissao, m_inicio, sq_1)

array([[1.00000000e+00, 2.54894249e+00, 6.53659037e+00, 1.63323716e+01,
        4.54688576e+01, 1.16064407e+02, 2.95820429e+02, 7.58841067e+02,
        1.89355858e+03, 5.29926375e+03, 8.43831807e+03],
       [3.91761930e-01, 1.00000000e+00, 2.54894249e+00, 6.53659037e+00,
        1.63323716e+01, 4.16893552e+01, 1.06265697e+02, 2.72490167e+02,
        6.81073528e+02, 1.89355858e+03, 3.13905432e+03],
       [1.53528579e-01, 3.91761930e-01, 1.00000000e+00, 2.54894249e+00,
        6.53659037e+00, 1.66851252e+01, 4.25292446e+01, 1.09065329e+02,
        2.72490167e+02, 7.58841067e+02, 1.24549575e+03],
       [6.01620031e-02, 1.53528579e-01, 3.91761930e-01, 1.00000000e+00,
        2.54894249e+00, 6.50635435e+00, 1.65843385e+01, 4.25292446e+01,
        1.06265697e+02, 2.95820429e+02, 4.86654680e+02],
       [2.35756298e-02, 6.01620031e-02, 1.53528579e-01, 3.91761930e-01,
        1.00000000e+00, 2.55257081e+00, 6.50635435e+00, 1.66851252e+01,
        4.16893552e+01, 1.16064407e+02, 1.90834251e+

## Questão 5

> Consider the model of _Question 4_ to answer the questions below you may either implement the appropriate recursions your learned in class or **use the MTK module of the Tangram-II tool** (the manual is available).

1. **Suppose you observe the sequence $HTHHT$.**

In [ ]:
seq_5_1 = 'HTHHT'

a) _What is the most probable sequence of coins if you assign the initial state probabilities as $\langle 0.5,0.5 \rangle$ for coin $A$ and $B$, respectively._

Para calcular a probabilidade de uma sequência de observações ter sido gerada por uma sequência de amostras, basta multiplicar:

- Probabilidade do estado inicial; pela:
- Probabilidade de transição entre o estado anterior e atual; e
- Probabilidade de emissão deste estado para a observação no $t$ atual.

E ir repetindo os $2$ últimos passos (pois a probabilidade do estado inicial basta apenas uma vez) até chegar ao final da sequência.

Como por exemplo, supondo que queremos calcular a probabildiade da sequência observada ($H \rightarrow T$) ter sido gerada pela sequência de estados ($A \rightarrow B$), para isso devemos multiplicar:

- Probabilidade de iniciar com o primeiro da amostra (neste caso $A$, que tem a mesma probabilidade de início de $B$) $= 0.5$
- Probabilidade deste estado inicial ($A$) emitir a amostra visível ($H$) em $t_1$ $= 0.9$
- Probabilidade de transacionar entre estados (trocar da moeda $A$ para $B$) $= 0.7$
- Probabilidade deste estado atual ($B$) emitir a amostra visível ($T$) em $t_2$ $= 0.9$

Multiplicando estes valores ($0.5 * 0.9 * 0.7 * 0.9$) teremos a probabilidade da amostra $HT$ ter sido gerada pela sequência de estados $AB$ que é $0.2835$

Portanto, se quiseremos saber a sequência de estados que melhor produz a sequência de observações, devemos efetuar o cáculo descrito acima para cada uma das possíveis combinações de estados, onde, neste exemplo, como temos $2$ estados (moeda $A$ e moeda $B$) e $2$ observações ($H$ e $T$), a quantidade de combinações possíveis será $2^2 = 4$.

Já para as sequências dadas pelo problema, ambas possuem $11$ amostras coletadas, logo, há $2^{11} = 2048$ diferentes combinações de estados possíveis que poderiam descrever tais amostra, cada um com sua probabilidade, e a que tiver o maior, será a mais provável.

Nota-se que seria extremamente custoso calcular a probabilidade de todas, pois é um problema que cresce exponecialmente. Para tanto, há uma forma menos custosa e que obtém o resultado precisando realizar menos cálculos, que é o algoritmo de _Viterbi_:

In [ ]:
def viterbi(T, E, pi, amostra):
    
    probabilities = []
    if amostra[0] == 'H':
        probabilities.append((pi[0] * E[0,0], pi[1] * E[1, 0]))
    else:
        probabilities.append((pi[0] * E[0,1], pi[1] * E[1, 1]))
    
    for i in range(1,len(amostra)):
        prev_A, prev_B = probabilities[-1]
        
        if amostra[i] == 'H':
            now_A = max(prev_A * T[0,0] * E[0,0], prev_B * T[1,0] * E[0,0])
            now_B = max(prev_A * T[0,1] * E[1,0], prev_B * T[1,1] * E[1,0])
            probabilities.append((now_A, now_B))
        else:
            now_A = max(prev_A * T[0,0] * E[0,1], prev_B * T[1,0] * E[0,1])
            now_B = max(prev_A * T[0,1] * E[1,1], prev_B * T[1,1] * E[1,1])
            probabilities.append((now_A, now_B))

    coins = []
    probs = []
    for p in probabilities:
        if p[0] > p[1]:
            coins.append('A')
            probs.append(p[0])
        else:
            coins.append('B')
            probs.append(p[1])

    return coins, probs

In [ ]:
prob_seq_1 = viterbi(m_transicao, m_emissao, p_inicio, seq_5_1)
print('\nA sequência de moedas que melhorer descreve a sequência', seq_5_1, 'é:\n', *prob_seq_1[0])

b) _What is the most likely coin chosen in the third toss? $A$ or $B$, can you answer this question with confidence? Explain._

2. **Suppose you observe the sequence $HTHTHTHTHTH$**

In [ ]:
seq_5_2 = 'HTHTHTHTHTH'

a) _What is the most probable sequence of coins if you assign the initial state probabilities as $\langle0.5,0.5\rangle$ for coin $A$ and $B$, respectively._

In [ ]:
prob_seq_2_a = viterbi(m_transicao, m_emissao, p_inicio, seq_5_2)
print('\nA sequência de moedas que melhorer descreve a sequência', seq_5_2, 'é:\n', *prob_seq_2_a[0])

b) _Repeat the item above if the initial state probabilities are $\langle0.2, 0.8\rangle$_

In [ ]:
new_pi = [0.2, 0.8]
prob_seq_2_b = viterbi(m_transicao, m_emissao, new_pi, seq_5_2)
print('\nA sequência de moedas que melhorer descreve a sequência', seq_5_2,
      'com probabilidade inicial', new_pi, 'é:\n', *prob_seq_2_b[0])

c) _Compare the likelihood of the sequence in both models above (i.e., models with different initial state probabilities). Which model is the most appropriate to explain the observed sequence?_

In [ ]:
from functools import reduce

likelihood_2_a = reduce(lambda x, y: x * y, prob_seq_2_a[1])
likelihood_2_b = reduce(lambda x, y: x * y, prob_seq_2_b[1])

print('\nLikelihood da sequência', seq_5_2,
     'com probabilidade inicial', p_inicio, 'é:\n', likelihood_2_a)
print('\nLikelihood da sequência', seq_5_2,
     'com probabilidade inicial', new_pi, 'é:\n', likelihood_2_b)
print('\nLogo, o modelo mais apropriado para explicar a observação', seq_5_2,
      'é o com probabilidade inicial:', p_inicio if likelihood_2_a > likelihood_2_b else new_pi)

## Questão 6

> [Dendroclimatology](http://en.wikipedia.org/wiki/Dendroclimatology) is the science of inferring past climates from trees, observing the properties of the annual tree rings. Studies have shown that there is a strong correlation among the size of a tree ring and the expected annual temperature. Scientist have used these findings to calculate the temperatures from thousands of years in the past. Since there is no temperature records from distance past, mathematical models such as that used below are very useful. In this question, our purpose is to use HMMs in this application.

> First you need to build a HMM. For that, assume that you collected data on tree rings of many trees and have also available the temperature of each year in the recent past (say the last 50 years). To make this example simple, we assume that there are only two temperature levels: $C = cold$ and $H = hot$. In addition, we measure only three different ring sizes: $S = small$ and $M = medium$ and $L = large$.
The following table lists the probabilities of the sizes of tree rings according to the temperature:

<img src="./imgs/table.png" width="400"/>

> An important observation is that recent data indicates that the sequence of temperatures at each year are correlated with that of previous year. For our problem assume that the probability of a hot (cold) year followed by another hot (cold) year is $0.75$ (respectively $0.6$)

**1. If there was _no correlation_ at all between the temperatures from one year to another, what would be the most probable sequence of temperatures if you observe a sequence of rings _SMSL_?**

Seria a temperatura mais právovel para cada observação:

- $S \rightarrow C = 0.8$
- $M \rightarrow H = 0.4$
- $L \rightarrow H = 0.55$

Logo, para a sequência $SMSL$ a melhor sequência de temperatura seria $CHCH$

**2. Well, you know that the temperatures are correlated from one year to another, and you need to build a HMM! What would be the choice for the hidden states? Note that you have available the size of tree rings from the distant past, but not the temperatures!**

- Se não possuimos as temperaturas, logo, estas serão os estados ocultos, formando a matriz de transição.
- Se possumos os aneis das árvores, logo, estas serão os estados visíveis, formando a matriz de emissão.

<img src="./imgs/hmm-6.png" width="400"/>

In [ ]:
#     [   H,    C]
h_t = [0.75, 0.15]
c_t = [0.40, 0.60]

#     [   S,    M,    L]
h_e = [0.05, 0.40, 0.55]
c_e = [0.80, 0.10, 0.10]

m_trasicao = np.array([h_t, c_t])
m_emissao  = np.array([h_e, c_e])
p_inicial  = matriz_pi(2)

**3. We have already given you a model, but you should briefly describe the necessary steps to build such model. That is, what sequences you need if no model is available to you? Are you solving problem $1, 2$ or $...$?**

Se o modelo não fosse já dado, seria preciso obter uma sequência onde, para cada ano, tivesse a temperatura e a largura do anel das árvores, pois desta forma, bastaria calcular a frequência de transição entre estados, e a frequência de cada emissão associada ao estado, para gerar as matrizes necessárias do modelo.

Quanto ao Problema, considerando que é sabido:
- Matriz de Transição
- Matriz de Emissão
- Probabilidade Inicial

Logo concluímos que temos, além das observações $\mathcal{O}_T$, também o modelo $\mathcal{M}$. Isso já elimina os Problemas $3$ e $4$, pois nestes devemos construir um modelo. Agora, dentre os problemas $1$ e $2$ o que se adequa à esta questão é o $2$, pois queremos descobrir, a partir das observações, a sequência de temperatura que melhor descreve tais dados coletados.

4. **Suppose you have the model given above. The only thing missing is the initial state probabilities, but assume identical probabilities for each hidden state. Suppose you are interested in the temperatures of the four years that precede those for which you have temperature data.**


a) _Calculate the probability of **each possible temperature sequence of length four**, when the sequence of ring sizes observed is **SMSL**._

In [ ]:
prob_amostra(m_transicao, m_emissao, p_inicio, )[0]

b) _From the $16$ results, calculate the most likely temperature at each year, independently of the other years._

c) _Using Tangram-II’s MTK, or an appropriate software package for HMMs, calculate the most probable sequence of temperature in these last four years, if the sequence of ring sizes observed is **SMSL**. Is this sequence identical to that of the previous item?_

In [ ]:
viterbi(m_trasicao, m_emissao, p_inicial, 'SMSL')

 = np.array([h_t, c_t])
  = np.array([h_e, c_e])
  = matriz_pi(2)

In [ ]:
def viterbi_arvore(T, E, pi, amostra):
    
    probabilities = []
    if amostra[0] == 'H':
        probabilities.append((pi[0] * E[0,0], pi[1] * E[1, 0]))
    else:
        probabilities.append((pi[0] * E[0,1], pi[1] * E[1, 1]))
    
    for i in range(1,len(amostra)):
        prev_A, prev_B = probabilities[-1]
        
        if amostra[i] == 'H':
            now_A = max(prev_A * T[0,0] * E[0,0], prev_B * T[1,0] * E[0,0])
            now_B = max(prev_A * T[0,1] * E[1,0], prev_B * T[1,1] * E[1,0])
            probabilities.append((now_A, now_B))
        else:
            now_A = max(prev_A * T[0,0] * E[0,1], prev_B * T[1,0] * E[0,1])
            now_B = max(prev_A * T[0,1] * E[1,1], prev_B * T[1,1] * E[1,1])
            probabilities.append((now_A, now_B))

    coins = []
    probs = []
    for p in probabilities:
        if p[0] > p[1]:
            coins.append('A')
            probs.append(p[0])
        else:
            coins.append('B')
            probs.append(p[1])

    return coins, probs

#### Problem 1
Given the observation sequence $\mathcal{O}_T = O_1, O_2, ..., O_T$ and a model $\mathcal{M}$, compute the probabilty of observing the output sequence $\mathcal{O}_T$ given the underlying model $\mathcal{M}$, i.e. $P[\mathcal{O}_T \mid \mathcal{M}]$.

#### Problem 2
Given the observation sequence $\mathcal{O}_T = O_1, O_2, ..., O_T$ and a model $\mathcal{M}$, how to determine a best state sequence $\mathcal{Q}_T = q_1, ..., q_T$ which _best explains_ the output sequence $\mathcal{O}_T$.

#### Problem 3
Given the observation sequence $\mathcal{O}_T = O_1, O_2, ..., O_T$, construct an underlying model $\mathcal{M}$, such that $P[\mathcal{O}_T \mid \mathcal{M}]$ is maximized.

#### Problem 4
Given several possible models $\mathcal{M}_i$ where $1 \leq i \leq K$ and a sequence of observed values, what is the probability that $\mathcal{M}_j$ is the actual model, i.e. $P[\mathcal{M}_j \mid \mathcal{O}]$.

In [ ]:
import numpy as np

a = [0.7, 0.1, 0.2]
b = [0.4, 0.5, 0.1]
c = [0.1, 0.1, 0.8]

transicao = np.array([a, b, c])
inicio = np.array([0.5, 0.3, 0.2])

tempo = 3
resultado = inicio.dot(np.linalg.matrix_power(transicao, tempo))

print(resultado * 100, '%')